<a href="https://colab.research.google.com/github/diptoromeo/LangChain-based-LLM-for-ChatGPT/blob/main/07_LangChain_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install langchain
!pip install langchain-community langchain-core
!pip install tiktoken
!pip install sentence_transformers
!pip install pypdf
!pip install chromadb #vectore store DB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nv

In [2]:
#API 접속 및 라이센스
import os

os.environ['OPENAI_API_KEY'] = '34fcaca1b0b04ebc935168215a7b44e8'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://labuser01openai.openai.azure.com/'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [3]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')


def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [5]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

In [6]:
loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=tiktoken_len)

texts = text_splitter.split_documents(pages)

In [10]:
# HuggingFaceEmbeddings api 호출 및 모델 구축
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(model_name=model_name,
                           model_kwargs=model_kwargs,
                           encode_kwargs=encode_kwargs)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
db = Chroma.from_documents(texts, hf)

In [12]:
openai = AzureChatOpenAI(
    deployment_name='dev-gpt-35',
    temperature=0.3,
    max_tokens=100
    )

In [20]:
from re import search
qa = RetrievalQA.from_chain_type(
    llm=openai,
    chain_type='stuff',
    retriever=db.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 3, 'fetch_k': 10}
    ),
    return_source_documents=True #원본에 어디에서 데이터가 찾아서 결과를 나와는지 확인해 준다.
    )

In [21]:
query = '탄소중립 사회란?'
qa(query)

{'query': '탄소중립 사회란?',
 'result': '탄소중립 사회란 개인, 회사, 단체 등이 배출하는 탄소와 흡수하는 탄소량을 같게 해서 탄소의 순배출을 제로로 만드는 것을 말합니다. 이는 인간의 활동으로 인해 발생하는 온실가스 배출을 최소화하고 남은 온',
 'source_documents': [Document(metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨\n- 국제사회는 기후변화에 대응하기 위해 선진국에 의무를 부여하는 ‘교토의정서 ’ 채택(1997 년)에 이어, 선\n진국과 개도국이 모두 참여하는 ‘파리기후변화협정 ’을 2015년 12월 12일 채택함 \n- 이후 2016년 11월 4일 발효된 ‘파리기후변화협정 ’은 (세계 온실가스 배출량의 90% 이상을 차지하고 있\n는) 협정 참여 195개 당사국이 지구 평균온도를 산업화